In [3]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# Preparing for data set
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transf)


#print(trainset[0][0].size())

trainld = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testld = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
dataiter = iter(trainld)
images, labels = dataiter.next()

In [5]:
print(images.size())

torch.Size([50, 3, 8, 8])


# 같은 클래스 별 폴더 이미지 데이터 이용

In [6]:
# ./class/tiger and ./class/lion
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./data/class', transform=transf)
trainloader = DataLoader(trainset, batch_size = 2, shuffle=False, num_workers=2)
print(len(trainloader))
trainset[0][0].size()

3


torch.Size([3, 16, 24])

# 개인데이터사용

In [7]:
#import preprocessing
train_images = np.random.randint(256, size=(20,32,32,3))
train_labels = np.random.randint(2, size=(20,1))

#preprocessing ....
#train_mages, train_labels - preprocessing(train_images, train_labels)

class TensorData(Dataset):
    
    def __init__(self, x_data, y_data):
        self.x_data = torch.tensor(x_data, dtype=torch.float32)
        self.x_data = self.x_data.permute(0,3,1,2) # to NCHW
        self.y_data = torch.tensor(x_data, dtype=torch.int32)
        self.len = self.y_data.shape[0]
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len


In [8]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
train_data[0][0].size()

torch.Size([3, 32, 32])

In [63]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
images.size()

torch.Size([10, 3, 32, 32])

# MyDataset class

In [70]:
train_images = np.random.randint(256, size=(20,32,32,3))
train_labels = np.random.randint(2, size=(20,1))

class MyDataset(Dataset):
    
    def __init__(self, x_data, y_data, transform=None):
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def __len__(self):
        return self.len
    
class MyToTensor:
    def __call__(self, sample):
        inputs, lables = sample
        inputs = torch.tensor(inputs, dtype=torch.float32)
        lables = torch.tensor(lables, dtype=torch.int32)
        inputs = inputs.permute(2,0,1) # HWC -> CHW
        return inputs, labels
    
class LinearTensor:
    def __init__(self, slope=1, bias=0):
        self.slope = slope
        self.bias = bias
    
    def __call__(self, sample):
        inputs, labels = sample
        inputs = self.slope * inputs + self.bias
        return inputs, labels

In [71]:
trans = tr.Compose([MyToTensor(), LinearTensor(2,5)])
ds1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(ds1, batch_size = 10, shuffle = True)

In [72]:
first_data = ds1[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [75]:
dataiter1 = iter(train_loader1)
images1, labels1 = dataiter1.next()
#images1


# MyDataset + use tr.transform

In [81]:
train_images = np.random.randint(256, size=(20,32,32,3))
train_labels = np.random.randint(2, size=(20,1))

class MyDataset(Dataset):
    
    def __init__(self, x_data, y_data, transform=None):
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def __len__(self):
        return self.len 
class MyTransform:
    def __call__(self, sample):
        inputs, lables = sample
        inputs = torch.tensor(inputs, dtype=torch.float32)
        lables = torch.tensor(lables, dtype=torch.int32)
        inputs = inputs.permute(2,0,1) # HWC -> CHW
        
        transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor(), tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        final_output = transf(inputs)
        return final_output, labels


In [82]:
ds2 = MyDataset(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2, batch_size = 10, shuffle = True)

In [83]:
first_data = ds2[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [84]:
dataiter2 = iter(train_loader2)
images2, labels2 = dataiter2.next()
images2.size()

torch.Size([10, 3, 128, 128])